# Results Notebook for the SIR stochastic simulation algorithm(s)

This notebook contains the code for all the figures and results in a time varying birth rate sceanrio. We use the herpesvirus model for the birth rate and parameters.

In [1]:
# Load necessary libraries
import numpy as np
from scipy.stats import multinomial, skew
import math
import metavirommodel as mm
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Choose array of colours for graphs and compartments names
colours = ['blue', 'red', 'green', 'purple', 'orange', 'black', 'gray', 'pink']
compartments = ['S', 'I', 'R']

## Gillespie algorithm plot for varying environment

#### Define rodent population

In [ ]:
import datetime
datetime.date(*map(int, start_date.split('-')))

In [2]:
# Set initial reproduction number
R_0 = 5

# Set initial population state S - I - R
N_init = 400
# S_init = int(N_init / R_0)
S_init = 380
I_init = N_init - S_init
R_init = 0
initial_population = [S_init, I_init, R_init]

# Set birth rate
theta = mm.BirthRateSeason([0.5, 0.15])

# Set death rates
mu = 0.04
nu = 0

# Set transition rates
infect_period = 14
# beta = R_0 / infect_period
# gamma = 1 / infect_period

beta = 1 * 10**-3 * N_init
gamma = 0.25

# Coalesce into paramater vector
parameters = initial_population
parameters.extend([theta, mu, nu, beta, gamma])

# Instantiate algorithm
algorithm = mm.Metaviromodel()

# Select start and end times
start_time = 1
end_time = 360

times = list(range(start_time, end_time+1))

# Select number of experiments
num_experiments = 1

output_algorithm = []
I_history_algorithm = []
I_times_history_algorithm = []

for _ in range(num_experiments):
    output, I_history, I_times_history = algorithm.simulate_fixed_times(parameters, start_time, end_time)
    output_algorithm.append(output)
    I_history_algorithm.append(I_history)
    I_times_history_algorithm.append(I_times_history)

output_algorithm = np.asarray(output_algorithm)

### Plot output of Gillespie for the different compartments

In [3]:
# Trace names - represent the type of individuals for the simulation
trace_name = ['{}'.format(s) for s in compartments]

# Names of panels
panels = ['{} only'.format(s) for s in compartments]

fig = go.Figure()
fig = make_subplots(rows=int(np.ceil(len(panels)/2)), cols=2, subplot_titles=tuple('{}'.format(p) for p in panels))

# Add traces to the separate counts panels
for s, spec in enumerate(compartments):
    fig.add_trace(
        go.Scatter(
            y=np.mean(output_algorithm[:, :, s], axis=0).tolist(),
            x=times,
            mode='lines',
            name=trace_name[s],
            line_color=colours[s]
        ),
        row= int(np.floor(s / 2)) + 1,
        col= s % 2 + 1
    )

# Add axis labels
fig.update_layout(
    title='Counts of compartments over time:<br>IC = {}, θ = {}, μ = {}, v = {}, β = {:.2f}, γ = {:.2f}'.format(parameters[0:3], parameters[3], parameters[4], parameters[5], parameters[6], parameters[7]),
    width=1100, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        title = 'Time (days)'
        ),
    yaxis=dict(
        linecolor='black',
        title = 'Individuals'),
    xaxis2=dict(
        linecolor='black',
        title = 'Time (days)'
        ),
    yaxis2=dict(
        linecolor='black',
        title = 'Individuals'),
    xaxis3=dict(
        linecolor='black',
        title = 'Time (days)'
        ),
    yaxis3=dict(
        linecolor='black',
        title = 'Individuals')
    #legend=dict(
    #    orientation="h",
    #    yanchor="bottom",
    #    y=1.02,
    #    xanchor="right",
    #    x=1
    #)
    )

fig.write_image('images/SIR-gillespie-HPV.pdf')
fig.show()

In [4]:
# Trace names - represent the type of individuals for the simulation
trace_name = ['{}'.format(s) for s in compartments]

# Names of panels
panels = ['{} only'.format(s) for s in compartments]

fig = go.Figure()
fig = make_subplots(rows=int(np.ceil(len(panels)/2)), cols=2, subplot_titles=tuple('{}'.format(p) for p in panels))

# Add traces to the separate counts panels
for s, spec in enumerate(compartments):
    for _ in range(num_experiments):
        fig.add_trace(
            go.Scatter(
                y=output_algorithm[_, :, s].tolist(),
                x=times,
                mode='lines',
                name=trace_name[s],
                line_color=colours[s],
                showlegend=False,
            ),
            row= int(np.floor(s / 2)) + 1,
            col= s % 2 + 1
        )

# Add axis labels
fig.update_layout(
    title='Counts of compartments over time:<br>IC = {}, θ = {}, μ = {}, v = {}, β = {:.2f}, γ = {:.2f}'.format(parameters[0:3], parameters[3], parameters[4], parameters[5], parameters[6], parameters[7]),
    width=1100, 
    height=600,
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        title = 'Time (days)'
        ),
    yaxis=dict(
        linecolor='black',
        title = 'Individuals'),
    xaxis2=dict(
        linecolor='black',
        title = 'Time (days)'
        ),
    yaxis2=dict(
        linecolor='black',
        title = 'Individuals'),
    xaxis3=dict(
        linecolor='black',
        title = 'Time (days)'
        ),
    yaxis3=dict(
        linecolor='black',
        title = 'Individuals')
    #legend=dict(
    #    orientation="h",
    #    yanchor="bottom",
    #    y=1.02,
    #    xanchor="right",
    #    x=1
    #)
    )

fig.write_image('images/SIR-gillespie-separate-HPV.pdf')
fig.show()

### Plot incidence of infection

In [5]:
fig = go.Figure()

incidence = np.zeros((num_experiments, len(times)))
incidence[:, 1:] = output_algorithm[:, 1:, 1] - output_algorithm[:, :-1, 1]

for _ in range(num_experiments):
    fig.add_trace(
        go.Scatter(
            y=incidence[_, :].tolist(),
            x=times,
            mode='lines',
            name=trace_name[s],
            line_color=colours[s],
            showlegend=False,
        ),
)

# Add axis labels
fig.update_layout(
    width=500, 
    height=500,
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        title = 'Time (days)'
        ),
    yaxis=dict(
        linecolor='black',
        title = 'Individuals'),
    #legend=dict(
    #    orientation="h",
    #    yanchor="bottom",
    #    y=1.02,
    #    xanchor="right",
    #    x=1
    #)
    )

fig.write_image('images/Incidence-HPV.pdf')
fig.show()

## Produce Viral read counts values

In [6]:
# Set parameter for the viral read counts model
t_eclipse = 2  # (0 days) Time from infection to initial viral growth
t_peak = 5  # (5 days ) Time from initial viral growth to peak viral load
t_switch = 10  # (9.38 days) Time from peak viral load to secondary waning phase
t_mod = 14  # (14 days) time from secondary waning phase until gumbel distribution reaches its min scale parameter
t_LOD = math.inf  # ( inf days ) Time from infection until modal read counts value is equal to the limit of detection

sigma_obs = 5  # Initial scale parameter for the Gumbel distribution until a=teclipse+tpeak+tswitch
s_mod = 0.4  # 0.4 multiplicative factor applied to scale paramter for the Gumble distrbution - starting at t_eclipse + t_peak + t_switch + t_scle
v_zero = 2  # read counts value at time of infection
v_peak = 2990  # (20) Modal read counts value at peak viral load
v_switch = 1078  # (33) Modal read counts value at a = teclipse + tpeak + tswitch
v_LOD = 18  # Limit of detection of read counts value

parameters_vl = [
    t_eclipse, t_peak, t_switch, t_mod, t_LOD,
    v_zero, v_peak, v_switch, v_LOD,
    s_mod, sigma_obs]

# Set read counts value for the suceptible and recovered individuals
VR_susc = 40
VR_rec = 40

### Sample individuals at specific points in time

In [7]:
# Sample indviduals on days 30, 80, 150 and 200
sample_points = np.arange(start_time, end_time, 30)

# Seelect sample size
sample_size = 50

vr_values = []
infec = []
time_since_infec = []

for _ in range(num_experiments):
    experiment_vr_values = []
    experiment_infec = []
    experiment_time_since_infec = []
    # At each point in time sample sample_size individuals
    for time in sample_points:
        # Identify the current infections at the specified timepoint
        current_infection_times = I_times_history_algorithm[_][time-1]

        # Sample without replacement the sample_size individuals and
        # determine their time since infection to produce Ct values
        number_selected_susc, number_selected_infec, number_selected_rec = \
            multinomial.rvs(
                n=sample_size,
                p=output_algorithm[_, time-1, :]/np.sum(output_algorithm[_, time-1, :])) # determine how many of those sampled are S, I and R
        
        if len(current_infection_times) > 0:
            # If we have at least one selected infection
            selected_individuals_infec_times = np.random.choice(
                current_infection_times,
                size=number_selected_infec,
                replace=False) # determine the time of infection of those sampled Is
        
            sample_time_since_infec = time - selected_individuals_infec_times # determine how long since infection for selected Is

        # First add the Ct values for the sampled susceptibele and recovered individuals
        sampled_vr_values = [VR_susc] * number_selected_susc + [VR_rec] * number_selected_rec

        # Run Ct model to determine individual Ct counts for each sample
        for ti in sample_time_since_infec:
            sampled_vr_values.append(algorithm.viral_read_model(parameters_vl, ti))

        experiment_vr_values.append(sampled_vr_values)
        experiment_infec.append(number_selected_infec)
        experiment_time_since_infec.append(sample_time_since_infec)
    
    vr_values.append(experiment_vr_values)
    infec.append(experiment_infec)
    time_since_infec.append(experiment_time_since_infec)

vr_values = np.asarray(vr_values)
infec = np.asarray(infec)

### Plot Ct values for different experiments

In [8]:
fig = go.Figure()
fig = make_subplots(rows=2, cols=1, subplot_titles=('Average Ct value', 'Average Prevalence of Infections'))

skews = skew(vr_values[0, :, :], axis=1)
print(skews)

for _ in range(sample_size):
    fig.add_trace(
        go.Scatter(
            y=vr_values[0, :, _].tolist(),
            x=sample_points,
            mode='markers',
            name='Average Ct value',
            marker_line=dict(width=1.25, color='black'),
            marker_color=skews,
            marker_colorscale='BlueRed',
            marker_colorbar_title=dict(
                text='Skew',
                side='top'),
            marker_size=12,
            marker_opacity=0.6,
            marker_showscale=True,
            showlegend=False,
        ),
        row=1,
        col=1
    )

fig.add_trace(
    go.Scatter(
        y=np.mean(vr_values[0, :, :], axis=1).tolist(),
        x=sample_points,
        mode='markers',
        name='Median Ct value',
        marker_color='black',
        marker_line=dict(width=4, color='black'),
        marker_symbol='line-ew',
        marker_size=16,
        showlegend=False,
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        y=(infec[0, :]/sample_size).tolist(),
        x=sample_points,
        mode='lines+markers',
        connectgaps=True,
        name='Average Prevalence of Infections',
        marker_line=dict(width=1.25, color='black'),
        marker_color=skews,
        marker_colorscale='BlueRed',
        marker_colorbar_title=dict(
            text='Skew',
            side='top'),
        marker_opacity=0.6,
        marker_size=12,
        marker_showscale=True,
        line_color='black',
        showlegend=False,
    ),
    row=2,
    col=1
)

# Add axis labels
fig.update_layout(
    width=1100, 
    height=800,
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        tickvals=sample_points.tolist(),
        title = 'Time (days)'
        ),
    yaxis=dict(
        linecolor='black',
        title = 'Viral read value (e+03)'),
    xaxis2=dict(
        linecolor='black',
        tickvals=sample_points.tolist(),
        title = 'Time (days)'
        ),
    yaxis2=dict(
        linecolor='black',
        tickformat = '.0%',
        title = 'Prevalence of Infections<br>in sample')
    #legend=dict(
    #    orientation="h",
    #    yanchor="bottom",
    #    y=1.02,
    #    xanchor="right",
    #    x=1
    #)
    )

fig.write_image('images/Viral_read_values-HPV.pdf')
fig.show()

[-3.11384004  2.79372712  2.33874508  2.33876118  1.78088573  2.07507929
  2.07501238  2.31867728  2.42666722  1.35187985  1.35128732  1.0943229 ]


### Plot skenewss and median

In [9]:
fig = go.Figure()
fig = make_subplots(rows=2, cols=1, subplot_titles=('Median time since infection', 'Skewness Ct values'))

median_ti = []

for ti in time_since_infec[0]:
    median_ti.append(np.median(ti))


fig.add_trace(
    go.Scatter(
        y=median_ti,
        x=sample_points,
        mode='lines+markers',
        connectgaps=True,
        name='Median times',
        marker_line=dict(width=1.25, color='black'),
        marker_opacity=0.6,
        marker_size=12,
        line_color='black',
        showlegend=False,
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        y=skews,
        x=sample_points,
        mode='lines+markers',
        connectgaps=True,
        name='Skewness viral read values',
        marker_line=dict(width=1.25, color='black'),
        marker_opacity=0.6,
        marker_size=12,
        line_color='black',
        showlegend=False,
    ),
    row=2,
    col=1
)

# Add axis labels
fig.update_layout(
    width=1100, 
    height=800,
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        tickvals=sample_points.tolist(),
        title = 'Time (days)'
        ),
    yaxis=dict(
        linecolor='black',
        title = 'Median Time since infection'),
    xaxis2=dict(
        linecolor='black',
        tickvals=sample_points.tolist(),
        title = 'Time (days)'
        ),
    yaxis2=dict(
        linecolor='black',
        title = 'Skewness viral read values')
    #legend=dict(
    #    orientation="h",
    #    yanchor="bottom",
    #    y=1.02,
    #    xanchor="right",
    #    x=1
    #)
    )

fig.write_image('images/Median_vs_Skewness-HPV.pdf')
fig.show()

In [10]:
np.mean(vr_values[0, :, :], axis=1)

array([ 36.94875675, 151.73839026, 164.46800678, 164.50844226,
       263.67470626, 185.08218422, 185.23375847, 225.34698538,
       306.12987285, 267.66034929, 268.84653528, 309.80725724])

In [11]:
skews

array([-3.11384004,  2.79372712,  2.33874508,  2.33876118,  1.78088573,
        2.07507929,  2.07501238,  2.31867728,  2.42666722,  1.35187985,
        1.35128732,  1.0943229 ])

In [12]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        y=np.mean(vr_values[0, :, :], axis=1).tolist(),
        x=skews,
        mode='markers',
        name='Mean Ct',
        marker_color=sample_points,
        marker_colorscale='BlueRed',
        marker_colorbar_title=dict(
            text='Sample time',
            side='top'),
        marker_showscale=True,
        showlegend=False,
    )
)

# Add axis labels
fig.update_layout(
    width=500, 
    height=500,
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        title = 'Skewness Ct values'
        ),
    yaxis=dict(
        linecolor='black',
        title = 'Mean Ct'),
    #legend=dict(
    #    orientation="h",
    #    yanchor="bottom",
    #    y=1.02,
    #    xanchor="right",
    #    x=1
    #)
    )

fig.write_image('images/Mean_vs_Skewness-HPV.pdf')
fig.show()

### Plot Viral read Model

In [13]:
time_from_infec = np.arange(1, 50)
vr_val = []

for ti in time_from_infec:
    ti_vr_val = []
    for _ in range(10000):
        ti_vr_val.append(algorithm.viral_read_model(parameters_vl, ti))
    vr_val.append(ti_vr_val)

vr_val = np.asarray(vr_val)

In [14]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        y=time_from_infec,
        x=np.mean(vr_val, axis=1),
        mode='lines',
        name='Mean Viral read',
        showlegend=False,
    )
)

fig.add_trace(
    go.Scatter(
        y=time_from_infec.tolist() + time_from_infec.tolist()[::-1],
        x=np.quantile(vr_val, 0.975, axis=1).tolist() + np.quantile(vr_val, 0.025, axis=1).tolist()[::-1],
        mode='lines',
        fill='toself',
        fillcolor='blue',
        line_color='blue',
        opacity=0.3,
        showlegend=False,
    )
)

# Add axis labels
fig.update_layout(
    width=500, 
    height=500,
    plot_bgcolor='white',
    xaxis=dict(
        linecolor='black',
        title = 'Mean Viral read',
        autorange='reversed'
        ),
    yaxis=dict(
        linecolor='black',
        title = 'Time since infection'),
    #legend=dict(
    #    orientation="h",
    #    yanchor="bottom",
    #    y=1.02,
    #    xanchor="right",
    #    x=1
    #)
    )

fig.write_image('images/Viral_read_model-HPV.pdf')
fig.show()